In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/valentin.lemaire/brainiac-1-temporal


In [3]:
from sota_paper_implementation.convertors.d2g22tgt import convert_d2g2_sample_to_tgt
from brainiac_temporal.metrics.utility import link_forecasting_metric
import numpy as np
from pathlib import Path
import torch

In [4]:
%cd sota_paper_implementation/D2G2

/home/valentin.lemaire/brainiac-1-temporal/sota_paper_implementation/D2G2


In [5]:
data_path = Path("dataset/insecta/")
originals = convert_d2g2_sample_to_tgt(data_path, "")

# --------- load data
adj = np.load(data_path / Path("adj.npy"), allow_pickle=True)
features = np.load(data_path / Path("feature.npy"), allow_pickle=True)

# --------- after loading dataset
# to torch
adj = torch.from_numpy(adj).float()
features = torch.from_numpy(features).float()
dataset = torch.utils.data.TensorDataset(adj, features)

(1, 39, 165, 165)
(1, 39, 165, 1)


In [6]:
from model import *
from trainer import *
import optuna

**You probably want to change 2 things:**

`device = torch.device('cpu')`
and
`epochs=1`

In [12]:
# batch_size = 8
# z_dim = 32
# genr_batch_size = 100
# f_dim= 256

from brainiac_temporal.metrics import MetricEvaluator


def objective(trial):
    batch_size = trial.suggest_int("batch_size", 8, 16, step=8)
    z_dim = trial.suggest_int("z_dim", 8, 64, step=8)
    genr_batch_size = trial.suggest_int("genr_batch_size", 100, 120, step=10)
    f_dim = trial.suggest_int("f_dim", 128, 512, step=64)

    seq_len = adj.size(1)
    max_num_nodes = adj.size(2)
    feature_dim = features.size(3)

    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True, num_workers=4
    )
    # device = torch.device('cuda:0')
    device = torch.device("cpu")

    d2g2 = D2G2(
        f_dim=f_dim,
        z_dim=z_dim,
        batch_size=batch_size,
        seq_len=seq_len,
        factorised=True,
        device=device,
        graphs=seq_len,
        feature_dim=feature_dim,
        max_num_nodes=max_num_nodes,
    )

    fix_f = torch.rand(f_dim, device=device)
    fix_f = fix_f.expand(genr_batch_size, seq_len, f_dim)
    test_f = fix_f

    trainer = Trainer(
        d2g2,
        loader,
        test_f,
        epochs=1,
        learning_rate=0.0002,
        device=device,
        max_num_nodes=max_num_nodes,
        genr_batch_size=genr_batch_size,
        original_full_adj=adj,
    )

    trainer.train_model()
    trainer.sample_graphs()

    samples = convert_d2g2_sample_to_tgt("output/")

    evaluator = MetricEvaluator(
        statistics=["degree"],
        utility_metrics=None,
        temporal_metrics=None,
    )

    results = evaluator(samples, originals)
    score = results["degree"]
    # score = link_forecasting_metric(samples, originals, max_epochs=10)
    return score

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=2)

[I 2023-09-21 14:30:48,516] A new study created in memory with name: no-name-87f518eb-98a8-4753-a266-53bd9f129793


Running Epoch : 1


1it [00:00,  1.63it/s]


Epoch 1 : Average Loss: 82525.59375 Edge Loss 1.3831706047058105 Node Loss 48882.38671875 KL of f : 72.19434356689453 KL of z : 33569.6328125 Bursty_Coeff: 0.0 Temporal_Efficiency: 0.0 Degree_Centrality 0.0 original_edges_total 78.0 recon_edges_total 529541.0
Training is complete
from sample_frames: edge and node:  torch.Size([120, 39, 40]) torch.Size([120, 39, 40])
from sample_frames: test_f_expand.shape:  torch.Size([120, 39, 448])
torch.Size([120, 39, 528]) torch.Size([120, 39, 528])
from sample_frames: recon.shape:  torch.Size([120, 39, 165, 165]) torch.Size([120, 39, 165, 1])
(120, 39, 165, 165)
(120, 39, 165, 1)


[I 2023-09-21 14:31:18,150] Trial 0 finished with value: 171.21349975980283 and parameters: {'batch_size': 16, 'z_dim': 40, 'genr_batch_size': 120, 'f_dim': 448}. Best is trial 0 with value: 171.21349975980283.


degree
Running Epoch : 1


1it [00:00,  1.16it/s]


Epoch 1 : Average Loss: 157697.5625 Edge Loss 1.379209041595459 Node Loss 74354.3125 KL of f : 55.547935485839844 KL of z : 83286.3203125 Bursty_Coeff: 0.0 Temporal_Efficiency: 0.0 Degree_Centrality 0.0 original_edges_total 78.0 recon_edges_total 534868.0
Training is complete
from sample_frames: edge and node:  torch.Size([100, 39, 32]) torch.Size([100, 39, 32])
from sample_frames: test_f_expand.shape:  torch.Size([100, 39, 384])
torch.Size([100, 39, 448]) torch.Size([100, 39, 448])
from sample_frames: recon.shape:  torch.Size([100, 39, 165, 165]) torch.Size([100, 39, 165, 1])
(100, 39, 165, 165)
(100, 39, 165, 1)


[I 2023-09-21 14:31:42,714] Trial 1 finished with value: 176.58903561659767 and parameters: {'batch_size': 16, 'z_dim': 32, 'genr_batch_size': 100, 'f_dim': 384}. Best is trial 0 with value: 171.21349975980283.


degree


In [14]:
import json

with open(data_path / Path("best_params.json"), "w") as f:
    f.write(json.dumps(study.best_params))